In [67]:
import pandas as pd
import Levenshtein
import nltk
from nltk.stem import RSLPStemmer

nltk.download('rslp')
ps = RSLPStemmer()
k_r = 0.68
k_d = 1
k_l = 3

[nltk_data] Downloading package rslp to /home/rodrigo/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


In [6]:
dataframe = pd.read_csv('data/qualificacoes.csv')

In [7]:
dataframe['norm_qual'] = dataframe['Nome'].apply(lambda nome: nome.replace('é', 'e').lower())
dataframe = dataframe.sort_values(by=['norm_qual'])

In [65]:
## Option 1
groups = list()

def put_in_group(norm_qual):
    stemmed_qual = ps.stem(norm_qual)

    found_group = True
    added = False
    for group in groups:
        for el in group:
            stemmed_el = ps.stem(el)
#             if len(el) <= k_l:
#                 ratio = Levenshtein.distance(stemmed_qual, stemmed_el)
#                 if ratio > k_d:
#                     found_group = False
#                     break
#             else:
            ratio = Levenshtein.ratio(stemmed_qual, stemmed_el)
            if ratio < k_r:
                found_group = False
                break
                
        if found_group:
            group.add(norm_qual)
            added = True
            break
        found_group = True
    if not added:
        groups.append(set([norm_qual]))
    
for line in dataframe.values:
    norm_qual = line[4]
    put_in_group(norm_qual)

# groups
        

In [68]:
## Option 2
groups = list()

def calculate(group, candidate):
    score = 0.0
    for el in group:
        stemmed_el = ps.stem(el)
        score += Levenshtein.ratio(candidate, stemmed_el)
    return score / len(group)

def put_in_group(norm_qual):
    stemmed_qual = ps.stem(norm_qual)
    
    best_group = None
    best_score = 0.0
    for group_candidate in groups:
        score_candidate = calculate(group_candidate, stemmed_qual)
        
        if score_candidate > k_r:
        
            if not best_group:
                best_group = group_candidate
                best_score = score_candidate
            else:
                if score_candidate > best_score:
                    best_group = group_candidate
                    best_score = score_candidate
    

    if not best_group:
        best_group = set([norm_qual])
        groups.append(best_group)
    else:
        best_group.add(norm_qual)
    
for line in dataframe.values:
    norm_qual = line[4]
    put_in_group(norm_qual)

groups

[{'a definir'},
 {'adm-terc.', 'fiadterc'},
 {'adv.(a/s)', 'recdo.(a/s)'},
 {'advogada', 'advogado'},
 {'agravado', 'agravante', 'agravdo', 'agravte'},
 {'apelado', 'apelante'},
 {'arremterc'},
 {'au'},
 {'autor'},
 {'credor'},
 {'cônjuge'},
 {'depositário'},
 {'embar'},
 {'embargado', 'embargante', 'embargaodo', 'embargdo', 'embargte'},
 {'empresa'},
 {'excipte', 'excpto', 'exectda', 'exectdo', 'executado'},
 {'exe', 'exeq', 'exeqte', 'exequente'},
 {'fisico'},
 {'gestor'},
 {'interessado'},
 {'invtarda'},
 {'litisativ.'},
 {'notificado', 'notificante'},
 {'outros'},
 {'perito'},
 {'pf'},
 {'proc.(a/s)(es)'},
 {'re', 'req', 'reqda', 'reqdo', 'reqt', 'reqte', 'requ', 'reque', 'reu'},
 {'recte.(s)'},
 {'repreleg'},
 {'reprtate'},
 {'requerente', 'requeri', 'requerida', 'requerido'},
 {'terceiro', 'terintcer'}]